### [1-1] 카카오 url 수집

# [1] 카카오 크롤링

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver #pip install selenium
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time
import selenium
from selenium.webdriver.common.action_chains import ActionChains
selenium.__version__ 
def collect_kakaomap_url(text):
    path = 'C:/chromedriver_win32/chromedriver.exe'
    source_url = "https://map.kakao.com/"
    driver = webdriver.Chrome(path)
    driver.get(source_url)
    # id="search.keyword.query" 태그 선택=> input 태그 
    searchbox=driver.find_element(By.ID,"search.keyword.query") 
    searchbox.send_keys(text) #url 수집을 원하는 값 입력
    # id="search.keyword.submit" 태그 선택=> input 태그 
    searchbutton=driver.find_element(By.ID,"search.keyword.submit") 
    #script를 실행
    #arguments[0].click(); : 첫번째 매개변수값을 클릭
    driver.execute_script("arguments[0].click();", searchbutton) #검색버튼클릭.
    time.sleep(2) #2초동안 대기 =>  검색 결과가 브라우저에 표시됨
    html = driver.page_source # 현재브라우저의 소스(html)
    soup = BeautifulSoup(html, "html.parser") # BeautifulSoup 분석
    # name="a": a태그
    # attrs={"class":"moreview" : a태그중에 class속성이 moreview인 태그
    moreviews = soup.find_all(name="a", attrs={"class":"moreview"}) # 상세보기들
    page_urls = [] 
    for moreview in moreviews:
        # moreview: 한개의 상세보기. a태그
        page_url = moreview.get("href") # a태그의 href 속성의 값
        page_urls.append(page_url) #상세보기의 href 속성값들 목록. 식당 URL 목록
    # driver.close()    
    more_place = driver.find_element(By.ID,'info.search.place.more')
    actions = ActionChains(driver )#스크롤내리기
    actions.move_to_element(more_place).perform()
    time.sleep(1)
    layer_button = driver.find_element_by_css_selector('body > div.coach_layer.coach_layer_type1 > div > div > div > span')
    layer_button.click()
    

    time.sleep(1)
    more_place.click()
    try:
        more_place.click()
    except:
        more_place = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/a')
        more_place.click()
    page_num = 3
    while True:
        time.sleep(1.5)
        html = driver.page_source # 현재브라우저의 소스(html)
        soup = BeautifulSoup(html, "html.parser") # BeautifulSoup 분석
        # name="a": a태그
        # attrs={"class":"moreview" : a태그중에 class속성이 moreview인 태그
        moreviews = soup.find_all(name="a", attrs={"class":"moreview"}) # 상세보기들
        
        for moreview in moreviews:
            # moreview: 한개의 상세보기. a태그
            page_url = moreview.get("href") # a태그의 href 속성의 값
            page_urls.append(page_url) #상세보기의 href 속성값들 목록. 강남역 고기집의 URL 정보 목록
        try:# 페이지 버튼마다 반복 없을 시 다음 페이지로 선택
            page_button = driver.find_element_by_css_selector('#info\.search\.page\.no{}'.format(page_num))
            page_num += 1
        except Exception:
            page_button = driver.find_element_by_css_selector('#info\.search\.page\.next')
            page_num = 2
            
        try:#버튼 클릭 및 요소가 없을 시 break
            page_button.click()
        except:
            break
    return page_urls

In [23]:
page_urls = collect_kakaomap_url('경기도 맛집')

In [8]:
page_urls

['https://place.map.kakao.com/10332413',
 'https://place.map.kakao.com/853028118',
 'https://place.map.kakao.com/1974266862',
 'https://place.map.kakao.com/27584230',
 'https://place.map.kakao.com/1065693087',
 'https://place.map.kakao.com/348276052',
 'https://place.map.kakao.com/27531028',
 'https://place.map.kakao.com/13559837',
 'https://place.map.kakao.com/1492599844',
 'https://place.map.kakao.com/220597413',
 'https://place.map.kakao.com/1677426767',
 'https://place.map.kakao.com/27540102',
 'https://place.map.kakao.com/211389635',
 'https://place.map.kakao.com/10785496',
 'https://place.map.kakao.com/1950123074',
 'https://place.map.kakao.com/8025375',
 'https://place.map.kakao.com/2029456983',
 'https://place.map.kakao.com/21359554',
 'https://place.map.kakao.com/1301973155',
 'https://place.map.kakao.com/7851288',
 'https://place.map.kakao.com/444516464',
 'https://place.map.kakao.com/745164647',
 'https://place.map.kakao.com/87323834',
 'https://place.map.kakao.com/21555096'

In [4]:
len(page_urls)

500

---
### [1-2] URL 1개에 대한 df생성
### TEST용 실제 실행하지 않아도 됨
---

In [21]:
# 상세보기에 조회된 고기집 목록을 조회
path = 'C:/chromedriver_win32/chromedriver.exe'
df = pd.DataFrame() # 컬럼만 조회
driver = webdriver.Chrome(path) # 브라우저 실행
# for page in page_urls:
df = pd.DataFrame()
time.sleep(1)
# for page in page_urls:
driver.get(page_urls[0]) # 식당 데이터 상세화면
time.sleep(2) # 2초 쉬기
html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
soup = BeautifulSoup(html, 'html.parser')


total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
try:
    tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
except:
    pass
# continue
if (int(tot_num.text)-3)%5 == 0:  
    more_text_num= ((int(tot_num.text)-3)//5)
else:
    more_text_num= ((int(tot_num.text)-3)//5)+1

if more_text_num == 0:
    pass
else:
    try:
        for i in range(more_text_num):
            more_text_in_store = driver.find_element_by_css_selector('span.txt_more')  
            actions = ActionChains(driver)#스크롤내리기
            actions.move_to_element(more_text_in_store).perform()
            more_text_in_store.click()
            time.sleep(0.5)
    except:
        pass
        
rate_list =[]
review_list =[]

html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
soup = BeautifulSoup(html, 'html.parser')
try:        
    store_name = soup.find(name = 'h2', attrs = {'class':'tit_location'})#가게이름
    addr = soup.find(name = 'span', attrs = {'class':'txt_address'})#가게주소
    store_operate_day = soup.find(name = 'span', attrs = {'class':'txt_operation'})#영업일 및 영업시간
    menu = soup.find_all(name = 'span', attrs = {'class':'loss_word'})
    
    img_urls = soup.find(name='span', attrs={'class':'bg_present'})
    img_urls = img_urls.get('style')
    start_idx = img_urls.find('url(') + 4  # "url(" 다음 인덱스부터 시작
    end_idx = img_urls.find(')', start_idx)
    url = img_urls[start_idx:end_idx]
    

    total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
    tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
    contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
    reviewer_average_score = contents_div.find_all(name="span", attrs={"class": "txt_desc"})#리뷰어의 평균평점
    #reviewer_num = soup.find(name="span", attrs={"class": "text_desc"})#리뷰어의 평균평점
    rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
    reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
except:
    pass
    #continue
    # try 구문에서 오류 발생된 경우 다음 페이지로 이동 
#     # 리뷰의 1페이지 정보를 df에 저장
menu_list = []
rate_list =[]
review_list =[]
reviewer_average_score_list = []
reviewer_num_list = []

# for i in range(reviewer_average_score):
#     reviewer_average_score_list.append(reviewer_average_score)
    
for i in range(len(rates)):
    rate_value = rates[i].find(name='span', attrs={"class": "ico_star"})
    style_value = rate_value.get('style')
    if style_value == 'width:20%;':
        rate = 1
    elif style_value == 'width:40%;':
        rate = 2
    elif style_value == 'width:60%;':
        rate = 3
    elif style_value == 'width:80%;':
        rate = 4
    elif style_value == 'width:100%;':
        rate = 5
    rate_list.append(rate)    
for review in reviews:
    rev_span = review.find(name="span")
    if rev_span is not None and rev_span.text:
        rev = rev_span.text
    else:
        rev = ''
    review_list.append(rev)
for i in range(len(reviewer_average_score)):
    if i%2 == 0:
        reviewer_average_score_list.append(reviewer_average_score[i].text)
    else:
        reviewer_num_list.append(reviewer_average_score[i].text)
menu_str = ''
if len(menu)>0:
    for i in menu:
        menu_str += i.text + ' '
    menu_list = menu_str.rstrip()
else:
    menu_list = ''       


store_name = store_name.text
try: 
    addr = addr.text
    addr = [i.strip() for i in addr.split('\n')]
    addr = ' '.join(addr)
except:
    addr = ''
try:
    store_operate_day = store_operate_day.text
    store_operate_day = store_operate_day.replace('\n', '')
except:
    store_operate_day = ''
#이미지 url 추가하기


df['리뷰'] = review_list
df['별점'] = rate_list
df['리뷰자 평균 평점'] = reviewer_average_score_list
df['리뷰자 누적 리뷰 개수'] = reviewer_num_list
df['영업 시간'] = store_operate_day
df['이미지url'] = url
df['가게명'] = store_name
df['가게 주소'] = addr
df['메뉴'] = menu_list
df.shape


(16, 9)

In [23]:
df

,리뷰,별점,리뷰자 평균 평점,리뷰자 누적 리뷰 개수,영업 시간,이미지url,가게명,가게 주소,메뉴
0,성발라가 오히려 망친 것인가..,1,501,3.8,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
1,조용할 시간에 가서 너무 맛있게 편하게 먹었습니다. 서비스에 대해선 호불호가 있을 ...,5,5,3.4,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
2,왠만하면리뷰안쓰는데 최악중에최악 정말최악사장님이글보면 저아시겠죠? 서비스는바라지도않...,1,1,1.0,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
3,생고기 소자 주문하려니 양이 적다고 중자주문 유도한다 둘이 왔는데 중짜? 얼마...,1,22,3.5,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
4,성시경이 띄워놓았지만 대구에는 이것보다 훨씬 가성비 좋고 맛있는 생고기 집이 많다 ...,2,41,3.2,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
5,가지마세요 식당 지저분함. 기본안주 더 달라고 해도 '주문이 밀려서요'라고 말하면서...,1,12,4.7,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
6,밑반찬이 많아서 좋았고 맛도 좋았습니다. 뭉티기도 처음 먹어보는데 쫄깃하고 회먹는 ...,4,7,3.7,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
7,생고기 끝내줘요,5,25,3.8,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
8,유튜브에 속은 기분. 생고기는 무난하나 굳이 이 가격에 이 웨이팅에? 먹을 정도는 ...,2,4,3.3,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이
9,생고기가 먼저 나오고 밑반찬이 세팅됩니다. 생고기 대자는 생각보다 양이 많지 않습니...,5,74,3.1,매일 17:00 ~ 24:00,'//t1.kakaocdn.net/thumb/T800x0.q80/?fname=htt...,참조은생고기육회,대구 북구 복현로 78 1층 (우)41528,생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이


In [22]:
menu_list

'생고기 (소) 탕탕육회 (중) 육회 (소) 육전 소고리짜글이'

---
### [1-3]반복문으로 돌리기
- 리뷰 없으면 continue
---

In [19]:
path = 'C:/chromedriver_win32/chromedriver.exe'
total_df = pd.DataFrame(columns=['리뷰','별점', '리뷰자 평균 평점','리뷰자 누적 리뷰 개수', '영업 시간','이미지url','가게명','가게 주소','메뉴', '카카오 url']) # 컬럼만 조회
driver = webdriver.Chrome(path) # 브라우저 실행
for i in page_urls:
    kakao_url = str(i)
    df = pd.DataFrame()
    time.sleep(1)
    # for page in page_urls:
    driver.get(i) # 식당 데이터 상세화면
    time.sleep(2) # 2초 쉬기
    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = BeautifulSoup(html, 'html.parser')
    total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
    try:
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
    except:
        continue
    
    if int(tot_num.text) == 0:#리뷰가 없을 시 다른 가게 탐색
        continue
    time.sleep(1)


    if (int(tot_num.text)-3)%5 == 0:  
        more_text_num= ((int(tot_num.text)-3)//5)
    else:
        more_text_num= ((int(tot_num.text)-3)//5)+1
        
    if more_text_num == 0:
        pass
    else:
        try:
            for i in range(more_text_num):
                more_text_in_store = driver.find_element_by_css_selector('span.txt_more')  
                actions = ActionChains(driver)#스크롤내리기
                actions.move_to_element(more_text_in_store).perform()
                more_text_in_store.click()
                time.sleep(0.5)
        except:
            continue



    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = BeautifulSoup(html, 'html.parser')
    try:        
        store_name = soup.find(name = 'h2', attrs = {'class':'tit_location'})#가게이름
        addr = soup.find(name = 'span', attrs = {'class':'txt_address'})#가게주소
        store_operate_day = soup.find(name = 'span', attrs = {'class':'txt_operation'})#영업일 및 영업시간
        menu = soup.find_all(name = 'span', attrs = {'class':'loss_word'})#메뉴
        
        img_urls = soup.find(name='span', attrs={'class':'bg_present'})
        img_urls = img_urls.get('style')
        start_idx = img_urls.find('url(') + 4  # "url(" 다음 인덱스부터 시작
        end_idx = img_urls.find(')', start_idx)
        url = img_urls[start_idx:end_idx]
        

        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        reviewer_average_score = contents_div.find_all(name="span", attrs={"class": "txt_desc"})#리뷰어의 평균평점
        #reviewer_num = soup.find(name="span", attrs={"class": "text_desc"})#리뷰어의 평균평점
        rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
    except:
        continue

    
    menu_list = []
    rate_list =[]
    review_list =[]
    reviewer_average_score_list = []
    reviewer_num_list = []


    for i in range(len(rates)):
        rate_value = rates[i].find(name='span', attrs={"class": "ico_star"})
        style_value = rate_value.get('style')
        if style_value == 'width:20%;':
            rate = 1
        elif style_value == 'width:40%;':
            rate = 2
        elif style_value == 'width:60%;':
            rate = 3
        elif style_value == 'width:80%;':
            rate = 4
        elif style_value == 'width:100%;':
            rate = 5
        rate_list.append(rate)    
    for review in reviews:
        rev_span = review.find(name="span")
        if rev_span is not None and rev_span.text:
            rev = rev_span.text
        else:
            rev = ''
        review_list.append(rev)
    for i in range(len(reviewer_average_score)):
        if i%2 == 0:
            reviewer_num_list.append(reviewer_average_score[i].text)
        else:
            reviewer_average_score_list.append(reviewer_average_score[i].text)

    store_name = store_name.text
    try: 
        addr = addr.text
        addr = [i.strip() for i in addr.split('\n')]
        addr = ' '.join(addr)
    except:
        addr = ''
    try:
        store_operate_day = store_operate_day.text
        store_operate_day = store_operate_day.replace('\n', '')
    except:
        store_operate_day = ''
        
    menu_str = ''
    if len(menu)>0:
        for i in menu:
            menu_str += i.text + ' '
        menu_list = menu_str.rstrip()
    else:
        menu_list = ''   
        
    #df생성
    df['리뷰'] = review_list
    df['별점'] = rate_list
    df['리뷰자 평균 평점'] = reviewer_average_score_list
    df['리뷰자 누적 리뷰 개수'] = reviewer_num_list
    df['영업 시간'] = store_operate_day
    df['이미지url'] = url
    df['가게명'] = store_name
    df['가게 주소'] = addr
    df['메뉴'] = menu_list
    df['카카오 url'] = kakao_url 
    
    total_df = pd.concat([total_df,df])
    print(total_df.shape)
    print(total_df.tail(10))

(529, 10)
                                                    리뷰 별점 리뷰자 평균 평점  \
519                       경치도 좋고 브런치도 너무 맛있어요 무조건 추천!!  5       5.0   
520         숙소 근처여서 브런치 먹으러 갔는데~ 맛도 있고뷰도 사랑스럽고 만족스러운곳!  5       4.9   
521          넓게 제주도의 뷰를 느낄 수 있어서 좋음!거기에 커피한잔 하면 진짜 끝내줌  5       4.9   
522  리뷰 이런거 잘 안쓰는데, 여기는 꼭 가볼만하다고추천하고 싶어서 짧지만 남겨봅니다....  5       4.9   
523      뷰도 멋지고 음료도 괜찮음! 여자친구가 굉장히 좋아함나름인생샷을 찍을 수 있는곳!  5       4.9   
524            여기 메뉴들은 하나같이 정성이 많이 들어가고비주얼이 예쁜것 같아요~!!  5       4.9   
525     뷰가 정말,, 최고에요아직 가오픈이지만 시설도 너무 좋고사장님들도 너무 친절 하세용  5       3.0   
526  제주의 맛과 경치에 취해 보세요이곳 one and only 에서...사장님 커피 너...  5       5.0   
527                                       뷰가 끝내줍니다.!!!  5       3.8   
528                                너무 좋은 경치에 커피맛도 좋네요~  5       5.0   

    리뷰자 누적 리뷰 개수             영업 시간  \
519            1  매일 09:00 ~ 20:00   
520           63  매일 09:00 ~ 20:00   
521           60  매일 09:00 ~ 20:00   
522           59  매일 09:00 ~ 20:00   
523           74  

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.147)


In [ ]:
total_df.to_csv('제주 카페.csv')

In [20]:
page_urls = collect_kakaomap_url('서울 맛집')

In [24]:
path = 'C:/chromedriver_win32/chromedriver.exe'
total_df = pd.DataFrame(columns=['리뷰','별점', '리뷰자 평균 평점','리뷰자 누적 리뷰 개수', '영업 시간','이미지url','가게명','가게 주소','메뉴', '카카오 url','업종']) # 컬럼만 조회
driver = webdriver.Chrome(path) # 브라우저 실행
for i in page_urls:
    kakao_url = str(i)
    df = pd.DataFrame()
    time.sleep(1)
    # for page in page_urls:
    driver.get(i) # 식당 데이터 상세화면
    time.sleep(2) # 2초 쉬기
    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = BeautifulSoup(html, 'html.parser')
    total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
    try:
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
    except:
        continue
    
    if int(tot_num.text) == 0:#리뷰가 없을 시 다른 가게 탐색
        continue
    time.sleep(1)


    if (int(tot_num.text)-3)%5 == 0:  
        more_text_num= ((int(tot_num.text)-3)//5)
    else:
        more_text_num= ((int(tot_num.text)-3)//5)+1
        
    if more_text_num == 0:
        pass
    else:
        try:
            for i in range(more_text_num):
                more_text_in_store = driver.find_element_by_css_selector('span.txt_more')  
                actions = ActionChains(driver)#스크롤내리기
                actions.move_to_element(more_text_in_store).perform()
                more_text_in_store.click()
                time.sleep(0.5)
        except:
            continue



    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = BeautifulSoup(html, 'html.parser')
    try:        
        store_name = soup.find(name = 'h2', attrs = {'class':'tit_location'})#가게이름
        addr = soup.find(name = 'span', attrs = {'class':'txt_address'})#가게주소
        store_operate_day = soup.find(name = 'span', attrs = {'class':'txt_operation'})#영업일 및 영업시간
        menu = soup.find_all(name = 'span', attrs = {'class':'loss_word'})#메뉴
        category_name = soup.find(name = 'span', attrs = {'class':'txt_location'})
        
        
        img_urls = soup.find(name='span', attrs={'class':'bg_present'})
        img_urls = img_urls.get('style')
        start_idx = img_urls.find('url(') + 4  # "url(" 다음 인덱스부터 시작
        end_idx = img_urls.find(')', start_idx)
        url = img_urls[start_idx:end_idx]
        

        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        reviewer_average_score = contents_div.find_all(name="span", attrs={"class": "txt_desc"})#리뷰어의 평균평점
        #reviewer_num = soup.find(name="span", attrs={"class": "text_desc"})#리뷰어의 평균평점
        rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
    except:
        continue

    
    menu_list = []
    rate_list =[]
    review_list =[]
    reviewer_average_score_list = []
    reviewer_num_list = []


    for i in range(len(rates)):
        rate_value = rates[i].find(name='span', attrs={"class": "ico_star"})
        style_value = rate_value.get('style')
        if style_value == 'width:20%;':
            rate = 1
        elif style_value == 'width:40%;':
            rate = 2
        elif style_value == 'width:60%;':
            rate = 3
        elif style_value == 'width:80%;':
            rate = 4
        elif style_value == 'width:100%;':
            rate = 5
        rate_list.append(rate)    
    for review in reviews:
        rev_span = review.find(name="span")
        if rev_span is not None and rev_span.text:
            rev = rev_span.text
        else:
            rev = ''
        review_list.append(rev)
    for i in range(len(reviewer_average_score)):
        if i%2 == 0:
            reviewer_num_list.append(reviewer_average_score[i].text)
        else:
            reviewer_average_score_list.append(reviewer_average_score[i].text)

    store_name = store_name.text
    try: 
        addr = addr.text
        addr = [i.strip() for i in addr.split('\n')]
        addr = ' '.join(addr)
    except:
        addr = ''
    try:
        store_operate_day = store_operate_day.text
        store_operate_day = store_operate_day.replace('\n', '')
    except:
        store_operate_day = ''
        
    menu_str = ''
    if len(menu)>0:
        for i in menu:
            menu_str += i.text + ' '
        menu_list = menu_str.rstrip()
    else:
        menu_list = ''   
    category_name = category_name.text
    category_name = category_name.replace('분류: ', '')
        
    #df생성
    df['리뷰'] = review_list
    df['별점'] = rate_list
    df['리뷰자 평균 평점'] = reviewer_average_score_list
    df['리뷰자 누적 리뷰 개수'] = reviewer_num_list
    df['영업 시간'] = store_operate_day
    df['이미지url'] = url
    df['가게명'] = store_name
    df['가게 주소'] = addr
    df['메뉴'] = menu_list
    df['카카오 url'] = kakao_url 
    df['업종'] = category_name
    
    total_df = pd.concat([total_df,df])
    print(total_df.shape)
    print(total_df.tail(10))

(280, 11)
                                                    리뷰 별점 리뷰자 평균 평점  \
270  사진 찍는 분위기는 정말로 좋아요.하지만 너무 산만한 분위기에 시끄럽고......1...  1       3.9   
271  음식이며 음료가격도 비싼데 모든지 셀프. 식기정리까지 고객이 다하라고함 파스타. 버...  1       3.7   
272  파주의 근래 가장 핫한 플레이스 중 하나입니다. 자유로에서 롯데 프리미엄 아울렛이 ...  4       3.7   
273  가야는이유 : 자리는 부족할리 없는, 한 자리에서 식사부터 음료까지 가능안가는이유 ...  3       3.3   
274  주차공간 협소해서 다른데에 차세우고 가셔야합니다.주말에 일찍 안가면 차는 못댄다고 ...  1       4.2   
275                                                     5       5.0   
276                                    커피주문한거 한시간만에 나옴  1       1.8   
277                 분위기, 맛 좋은데 테이블 더러움 물티슈로 슥 훔쳤는데 더러움  4       2.1   
278                                          넓고 인테리어 굿  5       4.5   
279                                       분위기 좋고 좌석 편함  5       3.6   

    리뷰자 누적 리뷰 개수             영업 시간  \
270            7  매일 09:00 ~ 22:00   
271            3  매일 09:00 ~ 22:00   
272          673  매일 09:00 ~ 22:00   
273        3,689  매일 09:00 ~ 22:00   
274            5  

In [22]:
total_df.to_csv('seoul.csv')